## 원본데이터 전처리
1. 도서 종류에 따라 데이터셋을 grouping, 분야 → 학년 → 학기 순서로 정렬
2. 학년 및 학기에 맞는 학생 분류 (예: 1학년 1학기, 4학년 2학기)
3. 도서별 대출 횟수 집계
4. 같은 사용자가 대출할 대출 횟수 가능성을 방지하여 도서 대출 학생 수 추가

In [7]:
import pandas as pd
import re
from datetime import datetime

# CSV 불러오기
df = pd.read_csv("../backend/data/BookLoan_10years_data.csv", encoding="utf-8")

# 날짜 변환
df["대출일자"] = pd.to_datetime(df["대출일자"])

# 입학년도 추론 (학번 앞 2자리)
df["입학년도"] = 2000 + df["학번"].astype(str).str[:2].astype(int)

# 현재 학년 (최대 4학년)
df["재학년차"] = datetime.now().year - df["입학년도"]
df["학년"] = df["재학년차"].apply(lambda x: x if x <= 4 else 4)

# 학기 계산 (3~9월 = 1학기, 10~2월 = 2학기)
df["월"] = df["대출일자"].dt.month
df["학기"] = df["월"].apply(lambda m: 1 if 3 <= m <= 9 else 2)

# 청구기호 분류 매핑
def classify_subject(code):
    try:
         # 문자열에서 연속된 숫자 추출 (예: "657.3 ㄹ468ㅈ v" → "657")
        match = re.match(r"(\d{3})", str(code).strip())
        if not match:
            return "기타"
        num = int(match.group(1))  # 3자리 정수
    except:
        return "기타"

    if 0 <= num <= 99: return "총류"
    elif 100 <= num <= 199: return "철학"
    elif 200 <= num <= 299: return "종교"
    elif 300 <= num <= 399: return "사회과학"
    elif 400 <= num <= 499: return "자연과학"
    elif 500 <= num <= 599: return "기술과학"
    elif 600 <= num <= 699: return "예술"
    elif 700 <= num <= 799: return "언어"
    elif 800 <= num <= 899: return "문학"
    elif 900 <= num <= 999: return "역사"
    else: return "기타"

df["분류"] = df["청구기호"].apply(classify_subject)

# 분야 리스트
subjects = ["총류", "철학", "종교", "사회과학", "자연과학", "기술과학", "예술", "언어", "문학", "역사", "기타"]

# 추천 집계 함수
def recommend_books(year, semester, category, top_n=3):
    subset = df[(df["학년"] == year) & (df["학기"] == semester) & (df["분류"] == category)]

    if subset.empty:
        return pd.DataFrame()  # 빈 DF 반환

    book_counts = (
        subset.groupby("서명")
        .agg(
            대출횟수=("서명", "count"),
            대출학생수=("학번", "nunique")
        )
        .reset_index()
        .sort_values(by="대출학생수", ascending=False)
    )
    return book_counts.head(top_n)

# 전체 결과 저장
results = []

for grade in range(1, 5):       # 1~4학년
    for semester in [1, 2]:     # 1학기, 2학기
        for subject in subjects:
            result = recommend_books(grade, semester, subject)
            if not result.empty:
                result["학년"] = grade
                result["학기"] = semester
                result["분야"] = subject
                results.append(result)

# 리스트 → 하나의 DataFrame으로 합치기
final_df = pd.concat(results, ignore_index=True)

# 분야 → 학년 → 학기 순서로 정렬
final_df = final_df.sort_values(
    by=["분야", "학년", "학기"], 
    ascending=[True, True, True]
).reset_index(drop=True)

# CSV로 저장
final_df.to_csv("../backend/data/recommend_all.csv", index=False, encoding="utf-8-sig")

print("✅ 분야 기준으로 정렬된 recommend_all.csv 파일 저장 완료!")

✅ 분야 기준으로 정렬된 recommend_all.csv 파일 저장 완료!


# RAGAS 평가 데이터셋 생성하기
### 1. CSV 파일 -> RAG 평가 CSV 포멧으로 변환 
- 위에서 시행된 도서 추천 목록 csv파일을 RAG 평가용 데이터셋으로 반환
- 평가용 데이터셋을 통해 RAGAS 테스트 진행

### 2. 웹문서 데이터 기반 평가 데이터셋 생성
- 한성대학교 학술정보관 이용 세칙에 관한 웹 문서 로드
- 세칙에 맞게 웹문서에서 context 매핑

In [1]:
import pandas as pd
from langchain_community.document_loaders import WebBaseLoader
import bs4

# === 1. 추천 결과 불러오기 ===
df = pd.read_csv("../backend/data/recommend_all.csv", encoding="utf-8")

rec_rows = []

# === 2. 추천 도서 기반 질문 생성 ===
for (grade, semester, category), group in df.groupby(["학년", "학기", "분야"]):
    contexts = group["서명"].tolist()
    # ground_truth = 대출학생수 상위 3개
    ground_truth_top3 = group.head(3)["서명"].tolist()

    question = f"{grade}학년 {semester}학기 {category} 분야에서 추천할 도서는?"

    rec_rows.append({
        "question": question,
        "contexts": str(contexts),
        "ground_truth": str(ground_truth_top3),  # 리스트 그대로 CSV에 저장
        "source": "도서대출내역.csv"
    })

import requests
from bs4 import BeautifulSoup

# === 2. 웹 규정 데이터 기반 평가셋 생성 ===
url = "https://hsel.hansung.ac.kr/intro_data.mir"
resp = requests.get(url)
soup = BeautifulSoup(resp.text, "html.parser")

rule_div = soup.find("div", id="intro_rule")

contexts_rules = []
for h4 in rule_div.find_all("h4", class_="sub_title"):
    section_title = h4.get_text(strip=True)
    ul = h4.find_next_sibling("ul")
    if ul:
        items = [li.get_text(strip=True) for li in ul.find_all("li")]
        section_text = section_title + " " + " ".join(items)
        contexts_rules.append(section_text)

rule_questions = [
    "도서관 이용시간은 어떻게 돼?",
    "도서관의 휴관일은 어떻게 돼?",
    "학술정보관 재학생 대출기간은 어떻게 돼?",
    "학술정보관 교직원 대출기간은 어떻게 돼?"
]
rule_answers = [
    "1. 자료열람실 : 학기 중 평일 09:00～21:00 토요일 11:00 ~ 15:00 학기 중 방학 중 평일 10:00～16:00 토요일 휴관2. 일반열람실 : 연중 06:30～23:00",
    "1. 일요일, 2. 법정공휴일, 3. 개교기념일",
    "재학생(학부) 대출기간은 15일, 대학원은 30일",
    "교직원 대출기간은 직원 60일, 연구원 및 조교는 30일"
]

rule_rows = []
for q, gt in zip(rule_questions, rule_answers):
    rule_rows.append({
        "question": q,
        "contexts": str(contexts_rules),
        "ground_truth": gt,
        "source": "학술정보관규정"
    })
    
# === 3. 통합 데이터셋 ===
eval_df = pd.DataFrame(rec_rows + rule_rows)

# 저장
eval_df.to_csv("../backend/data/ragas_eval_dataset.csv", index=False, encoding="utf-8-sig")

print("✅ ragas_eval_dataset.csv 저장 완료!")
print(eval_df.head())

✅ ragas_eval_dataset.csv 저장 완료!
                     question  \
0  1학년 1학기 기술과학 분야에서 추천할 도서는?   
1    1학년 1학기 문학 분야에서 추천할 도서는?   
2  1학년 1학기 사회과학 분야에서 추천할 도서는?   
3    1학년 1학기 언어 분야에서 추천할 도서는?   
4    1학년 1학기 역사 분야에서 추천할 도서는?   

                                            contexts  \
0  ['강박에 빠진 뇌 :신경학적 불균형이 만들어낸 멈출 수 없는 불안', '백반증, ...   
1  ['(박완서 장편소설)그 남자네 집', '(잊지 않으려고 시작한 매일의 습관) 기록...   
2  ['(데이터로 공감하고 똑똑하게 의사결정하는) 데이터 드리븐 디자인씽킹', '디자인...   
3  ['(ETS TOEIC) 토익기출 파트별 voca :최신 기출 예문 독점 수록', ...   
4  ['강대국 국제정치의 비극', '사피엔스 :유인원에서 사이보그까지, 인간 역사의 대...   

                                        ground_truth      source  
0  ['강박에 빠진 뇌 :신경학적 불균형이 만들어낸 멈출 수 없는 불안', '백반증, ...  도서대출내역.csv  
1  ['(박완서 장편소설)그 남자네 집', '(잊지 않으려고 시작한 매일의 습관) 기록...  도서대출내역.csv  
2  ['(데이터로 공감하고 똑똑하게 의사결정하는) 데이터 드리븐 디자인씽킹', '디자인...  도서대출내역.csv  
3  ['(ETS TOEIC) 토익기출 파트별 voca :최신 기출 예문 독점 수록', ...  도서대출내역.csv  
4  ['강대국 국제정치의 비극', '사피엔스 :유인원에서 사이보그까지, 인간 역사의 대...  도서대출내역.csv  


## RAGAS 평가 단계
- 1. 평가할 데이터셋 불러오기
- 2. RAG 체인 준비 (평가할 RAG 파이프라인을 불러오기)
- 3. 배치 실행 (예상 답변 생성)
- 4. RAGAS 평가 실행

In [2]:
import ast
import pandas as pd
from datasets import Dataset
import sys, os

# backend/rag_core 경로 추가후 기존 RAG 파이프라인 함수 import
sys.path.append(os.path.abspath(".."))
from backend.rag_core.pipeline import ask

# 1. 평가 데이터셋 로드
df = pd.read_csv("../backend/data/ragas_eval_dataset.csv")

# 2. contexts가 문자열 형태일 경우 리스트로 변환
def convert_contexts(c):
    try:
        return ast.literal_eval(c) if isinstance(c, str) else c
    except:
        return [c]

df["contexts"] = df["contexts"].apply(convert_contexts)

# 3. ask() 실행해서 answer 생성
def get_answer(q):
    return ask(q)["answer"]

df["answer"] = df["question"].apply(get_answer)

# 4. HuggingFace Dataset으로 변환
eval_dataset = Dataset.from_pandas(df)

print(eval_dataset)

/Users/songseung-yun/Desktop/HSU_Library_Backend/venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



🚀 인덱스 준비 중...
📄 문서 225개 → 청크 226개
📁 기존 Chroma 인덱스 로드 (API 호출 없음)
🎯 RAG 파이프라인 준비 완료!

Dataset({
    features: ['question', 'contexts', 'ground_truth', 'source', 'answer'],
    num_rows: 82
})


In [3]:
from ragas import evaluate
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
)

result = evaluate(
    dataset=eval_dataset,
    metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall,
    ],
)

print(result)          # dict 형태
print(result.to_pandas())  # 표 형태

Evaluating:   0%|          | 1/328 [00:01<06:30,  1.19s/it]Exception raised in Job[6]: IndexError(list index out of range)
Exception raised in Job[14]: IndexError(list index out of range)
Evaluating:  34%|███▍      | 112/328 [00:39<01:19,  2.72it/s]Exception raised in Job[122]: IndexError(list index out of range)
Exception raised in Job[130]: IndexError(list index out of range)
Evaluating:  42%|████▏     | 138/328 [00:48<01:16,  2.48it/s]Exception raised in Job[146]: IndexError(list index out of range)
Exception raised in Job[150]: IndexError(list index out of range)
Evaluating:  61%|██████    | 199/328 [01:11<00:47,  2.69it/s]Exception raised in Job[210]: IndexError(list index out of range)
Exception raised in Job[214]: IndexError(list index out of range)
Evaluating:  93%|█████████▎| 306/328 [01:51<00:09,  2.24it/s]Exception raised in Job[318]: IndexError(list index out of range)
Exception raised in Job[322]: IndexError(list index out of range)
Evaluating: 100%|██████████| 328/328 [02

{'context_precision': 0.7378, 'faithfulness': 0.0285, 'answer_relevancy': 0.8102, 'context_recall': 0.9919}
                    user_input  \
0   1학년 1학기 기술과학 분야에서 추천할 도서는?   
1     1학년 1학기 문학 분야에서 추천할 도서는?   
2   1학년 1학기 사회과학 분야에서 추천할 도서는?   
3     1학년 1학기 언어 분야에서 추천할 도서는?   
4     1학년 1학기 역사 분야에서 추천할 도서는?   
..                         ...   
77    4학년 2학기 총류 분야에서 추천할 도서는?   
78            도서관 이용시간은 어떻게 돼?   
79            도서관의 휴관일은 어떻게 돼?   
80      학술정보관 재학생 대출기간은 어떻게 돼?   
81      학술정보관 교직원 대출기간은 어떻게 돼?   

                                   retrieved_contexts  \
0   [강박에 빠진 뇌 :신경학적 불균형이 만들어낸 멈출 수 없는 불안, 백반증, 발병부...   
1   [(박완서 장편소설)그 남자네 집, (잊지 않으려고 시작한 매일의 습관) 기록하기로...   
2   [(데이터로 공감하고 똑똑하게 의사결정하는) 데이터 드리븐 디자인씽킹, 디자인씽킹 ...   
3   [(ETS TOEIC) 토익기출 파트별 voca :최신 기출 예문 독점 수록, (한...   
4   [강대국 국제정치의 비극, 사피엔스 :유인원에서 사이보그까지, 인간 역사의 대담하고...   
..                                                ...   
77  [JAVA 객체 지향 디자인 패턴, (명품) Java programming, 알고리...   
78  [제 1 조 (목적) 본 세칙은 한성대학교

In [3]:
import pandas as pd

# ragas 평가 결과 DataFrame으로 변환
result_df = result.to_pandas()

# RAGAS결과 CSV파일로 저장
result_df.to_csv("ragas_query_split_result.csv", index=False, encoding="utf-8-sig")